# Quickstart Dashboard 入门指南

**English:** Quickstart Dashboard Getting Started Guide  
**日本語:** Quickstart Dashboard 入門ガイド

本笔记帮助你快速体验 `notebooks/quickstart_dashboard.py` 中提供的轻量级统计仪表盘框架。通过少量代码即可把 Git2Base 的分析结果渲染成可交互的项目/运行选择页面，并在“全部”及各技术栈标签页查看单次运行卡片，在“趋势分析”标签中对比历史指标。

**English:** This notebook helps you quickly experience the lightweight statistics dashboard framework provided in `notebooks/quickstart_dashboard.py`. With only a small amount of code you can render Git2Base analysis results into an interactive project/run selection page, view single-run cards on the “All” and individual tech stack tabs, and compare historical metrics within the “Trend Analysis” tab.  
**日本語:** このノートブックでは、`notebooks/quickstart_dashboard.py` で提供される軽量な統計ダッシュボードフレームワークを素早く体験できます。少しのコードだけで Git2Base の分析結果をプロジェクトと実行を選択できるインタラクティブなページとして表示し、「すべて」と各技術スタックのタブで単一実行カードを確認し、「トレンド分析」タブで履歴指標を比較できます。

## 1. 环境准备

**English:** 1. Environment preparation  
**日本語:** 1. 環境の準備

确保你已经在 `artifacts/` 目录中生成了至少一次分析运行的数据。

**English:** Make sure you have generated at least one analysis run inside the `artifacts/` directory.  
**日本語:** `artifacts/` ディレクトリ内に少なくとも1回分の分析結果が生成されていることを確認してください。

目录结构应类似：

**English:** The directory structure should look similar to the following.  
**日本語:** 典型的なディレクトリ構成は次のとおりです。

```
artifacts/
  your-project/
    runs/
      20240101-120000/
        data/
          commit_files.csv
          diff_results.csv
          ...
```

这些 CSV 文件会被仪表盘加载并转换成 Pandas DataFrame。

**English:** These CSV files will be loaded by the dashboard and converted into Pandas DataFrames.  
**日本語:** これらの CSV ファイルはダッシュボードによって読み込まれ、Pandas DataFrame に変換されます。

## 2. 引入仪表盘框架

**English:** 2. Import the dashboard framework  
**日本語:** 2. ダッシュボードフレームワークの読み込み

In [ ]:
from quickstart_dashboard import (
    DiffFileChangeCountStatistic,
    QuickstartDashboard,
    RecordCountStatistic,
    load_statistic_from_notebook,
)

## 3. 创建仪表盘实例

默认情况下，框架会从 `../artifacts` 读取数据。如果你的数据位于其他位置，可以使用 `RunDataLoader` 指定新的路径。

**English:** By default the framework reads data from `../artifacts`. If your data resides elsewhere, you can provide a custom path through `RunDataLoader`.  
**日本語:** 既定ではフレームワークは `../artifacts` からデータを読み込みます。別の場所にデータがある場合は `RunDataLoader` でカスタムパスを指定できます。

> 如果你的技术栈配置文件不在默认的 `config/config.yaml` 路径，可以通过 `config_path` 参数传入自定义 YAML。  
> **English:** If your tech stack configuration file is not at the default `config/config.yaml` path, pass a custom YAML file through the `config_path` parameter.  
> **日本語:** 技術スタックの設定ファイルが既定の `config/config.yaml` にない場合は、`config_path` パラメーターでカスタム YAML ファイルを指定してください。

In [ ]:
# Replace base_dir with the directory that stores your analysis results (relative to this notebook).
dashboard = QuickstartDashboard(base_dir="../artifacts")
# You can also pass the loader explicitly:
# loader = RunDataLoader(base_dir="/path/to/artifacts")
# dashboard = QuickstartDashboard(loader=loader)


## 4. 注册基础统计

`RecordCountStatistic` 会根据运行类型自动选择合适的 CSV 表并统计记录数，方便快速校验仪表盘。`DiffFileChangeCountStatistic` 针对 diff 运行统计新增、变更、删除文件数量，snapshot 运行会显示为 N/A。如果你还想查看字符区间分布，可在后续步骤通过加载器引入 `FileCharCountDistributionStatistic` 自定义统计。

**English:** `RecordCountStatistic` automatically chooses the appropriate CSV table based on the run type and reports the row count, which is handy for validating the dashboard quickly. `DiffFileChangeCountStatistic` reports the numbers of added, modified, and deleted files for diff runs while showing N/A for snapshot runs. If you also want to inspect character count buckets, load the custom `FileCharCountDistributionStatistic` in later steps.  
**日本語:** `RecordCountStatistic` は実行タイプに応じて適切な CSV テーブルを自動的に選択し、行数を集計するため、ダッシュボードの動作確認に便利です。`DiffFileChangeCountStatistic` は diff 実行で追加・変更・削除されたファイル数を算出し、snapshot 実行では N/A を表示します。文字数区間の分布を確認したい場合は、後続の手順でカスタム統計 `FileCharCountDistributionStatistic` をロードしてください。

In [ ]:
dashboard.register_statistic(
    RecordCountStatistic()
)
dashboard.register_statistic(
    DiffFileChangeCountStatistic()
)

## 5. 自定义你的统计卡片

在 `notebooks/custom_statistics/` 目录下创建专用的开发 notebook，使用其中的可视化单元调试统计逻辑。完成后，通过下面的加载器将 notebook 中的 `BaseStatistic` 子类引入到当前 quickstart 环境。

**English:** Create a dedicated development notebook inside `notebooks/custom_statistics/` and use its visualization cells to debug your statistic logic. After finishing the implementation, use the loader below to import the `BaseStatistic` subclasses from that notebook into the current quickstart environment.  
**日本語:** `notebooks/custom_statistics/` ディレクトリに開発用ノートブックを作成し、可視化セルを利用して統計ロジックを調整してください。実装が完了したら、以下のローダーでそのノートブック内の `BaseStatistic` サブクラスを現在のクイックスタート環境に読み込みます。

In [ ]:
# Use the loader to bring statistic classes from development notebooks into this session.
# Cells that are only for previewing can include a 'skip-loader' tag in metadata
# to avoid executing helper code when importing through the loader.
# If the notebook defines multiple statistic classes, pass class_name to choose one.
FileCharCountDistributionStatistic = load_statistic_from_notebook(
    "custom_statistics/file_char_count_distribution_statistic.ipynb"
)
UniqueAuthorsStatistic = load_statistic_from_notebook(
    "custom_statistics/unique_authors_statistic.ipynb"
)


In [ ]:
# Register custom statistics (add as many as you need)
dashboard.register_statistic(FileCharCountDistributionStatistic())
dashboard.register_statistic(UniqueAuthorsStatistic())


## 6. 展示仪表盘

运行下方单元格，即可在 Jupyter 中看到交互式仪表盘。当你新增或删除运行结果后，可调用 `dashboard.refresh()` 重新加载数据。

**English:** Run the cell below to display the interactive dashboard inside Jupyter. Whenever you add or remove run results, call `dashboard.refresh()` to load the latest data.  
**日本語:** 下のセルを実行すると、Jupyter 上でインタラクティブなダッシュボードが表示されます。実行結果を追加または削除したときは、`dashboard.refresh()` を呼び出して最新のデータを読み込んでください。

> 技术栈切换：仪表盘会读取 `config/config.yaml` 中的 `stacks` 配置，并在每个仓库下按技术栈拆分标签页。 可以在这些标签页中查看所选运行的单次结果；而“趋势分析”标签提供技术栈、统计种类、执行类型与运行筛选，用于对比历史趋势。  
> **English:** Tech stack switching: the dashboard reads the `stacks` configuration from `config/config.yaml` and creates tabs per stack under each repository. Use these tabs to inspect single-run results for the selected run, while the “Trend Analysis” tab offers stack, statistic, execution type, and run filters for comparing historical trends.  
> **日本語:** 技術スタックの切り替え: ダッシュボードは `config/config.yaml` の `stacks` 設定を読み込み、各リポジトリの下にスタックごとのタブを作成します。これらのタブでは選択した実行の単一結果を確認でき、「トレンド分析」タブではスタック・統計種類・実行タイプ・実行のフィルターを使って履歴トレンドを比較できます。

In [ ]:
dashboard.show()
# Call dashboard.refresh() if you need to reload data
# dashboard.refresh()
